In [ ]:
!pip install natsort

In [ ]:
import numpy as np 
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from natsort import natsorted, ns
import h5py
import sklearn as sk
from skimage.transform import resize
from numpy import load
from numpy import zeros
from numpy import ones

In [ ]:
mypath = "../input/figshare-brain-tumor-dataset/dataset/data"
filelist = natsorted([f for f in listdir(mypath) if isfile(join(mypath, f))])

In [ ]:
filelist

In [ ]:
filelist100 = filelist[0:100]

In [ ]:
filelist100

In [ ]:
path = "../input/figshare-brain-tumor-dataset/dataset/data/"
datalist = []
for i in range(0,100):
    f = h5py.File(path + filelist100[i],'r')
    list(f.keys())
    datalist.append(np.array(f.get('cjdata/tumorMask/')))
    datalist[i].reshape(512,512,1)
    datalist[i] = resize(datalist[i], (256, 256))
    
np.array(datalist)


datalist

In [ ]:
plt.imshow(datalist[0])

In [ ]:
import tensorflow as tf


In [ ]:
kaamKaDataStack = tf.stack(
   datalist, axis=0, name='stack'
)

In [ ]:
kaamKaDataStack

In [ ]:
from numpy import savez

In [ ]:
np.savez("./test.npz", kaamKaDataStack)

In [ ]:
path = "../input/figshare-brain-tumor-dataset/dataset/data/"
imageDatalist = []
for i in range(0,100):
    f1 = h5py.File(path + filelist100[i],'r')
    list(f1.keys())
    imageDatalist.append(np.array(f1.get('cjdata/image/')))
    imageDatalist[i].reshape(512,512,1)
    imageDatalist[i] = resize(imageDatalist[i], (256, 256))
    
np.array(imageDatalist)

In [ ]:
plt.imshow(imageDatalist[0])

In [ ]:
import tensorflow as tf

In [ ]:
kaamKaImageStack = tf.stack(
   imageDatalist, axis=0, name='stack1'
)

In [ ]:
np.savez("./testimg.npz", kaamKaDataStack)

In [ ]:
pwd

In [ ]:
data_source2 = "./test.npz"
data_source1 = "./testimg.npz" 

In [ ]:

# defining a 70x70 patchgan discriminator model
from tensorflow.keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import BatchNormalization
from keras.utils.vis_utils import plot_model

# define the discriminator model
def define_discriminator(image_shape):
     # weight initialization
    init = RandomNormal(stddev=0.02)
	# source image input
    in_src_image = Input(shape=image_shape)
	# target image input
    in_target_image = Input(shape=image_shape)
	# concatenate images channel-wise
    merged = Concatenate()([in_src_image, in_target_image])
	# C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
	# C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
	# C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
	# C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
	# second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
	# patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
	# define model
    model = Model([in_src_image, in_target_image], patch_out)
	# compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

# define image shape
image_shape = (256,256,1)
# create the model
model = define_discriminator(image_shape)
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='discriminator_model_plot.png', show_shapes=True, show_layer_names=True)

generator model


In [ ]:
from keras.initializers import RandomNormal
from keras.models import Model
from keras.models import Input
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.utils.vis_utils import plot_model
from random import randint

In [ ]:
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    init = RandomNormal(stddev=0.02)
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [ ]:
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
    init = RandomNormal(stddev=0.02)
	# add upsampling layer
    g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
	# add batch normalization
    g = BatchNormalization()(g, training=True)
	# conditionally add dropout
    if dropout:
        g = Dropout(0.5)(g, training=True)
	# merge with skip connection
    g = Concatenate()([g, skip_in])
	# relu activation
    g = Activation('relu')(g)
    return g

In [ ]:
def define_generator(image_shape=(256,256,1)):
	# weight initialization
    init = RandomNormal(stddev=0.02)
	# image input
    in_image = Input(shape=image_shape)
	# encoder model: C64-C128-C256-C512-C512-C512-C512-C512
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
	# bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
	# decoder model: CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
    d1 = decoder_block(b, e7, 512)
    d2 = decoder_block(d1, e6, 512)
    d3 = decoder_block(d2, e5, 512)
    d4 = decoder_block(d3, e4, 512, dropout=False)
    d5 = decoder_block(d4, e3, 256, dropout=False)
    d6 = decoder_block(d5, e2, 128, dropout=False)
    d7 = decoder_block(d6, e1, 64, dropout=False)
	# output
    g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7)
    out_image = Activation('tanh')(g)
	# define model
    model = Model(in_image, out_image)
    return model
image_shape = (256,256,1)
# create the model
model = define_generator(image_shape)
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file='generator_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
def define_gan(g_model, d_model, image_shape):
	# make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
	# define the source image
    in_src = Input(shape=image_shape)
    print(in_src)
	# connect the source image to the generator input
    gen_out = g_model(in_src)
    #tf.reshape(gen_out, [-1,256,256,1])
    #print("I:",in_src.shape)
    #print("Gen:",gen_out.shape)
	# connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out])
	# src image as input, generated image and classification output
    model = Model(in_src, [dis_out, gen_out])
	# compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model
image_shape = (256,256,1)
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)

In [ ]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples, patch_shape):
	# unpack dataset
    trainA, trainB = dataset
	# choose random instances
    #print(trainA.shape)
    ix = np.random.randint(0, trainA.shape[0], n_samples)
	# retrieve selected images
    X1, X2 = trainA[ix], trainB[ix]
	# generate 'real' class labels (1)
    y = ones((n_samples, patch_shape, patch_shape, 1)) 
    return [X1, X2], y


In [ ]:
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
    X = g_model.predict(samples)
	# create 'fake' class labels (0)
    y = zeros((len(X), patch_shape, patch_shape, 1))
    return X, y

In [ ]:
def summarize_performance(step, g_model, dataset, n_samples=3):
	# save the generator model
	filename2 = 'model_%06d.h5' % (step+1)
	g_model.save(filename2)
	print('>Saved:  %s' % (filename2))

In [ ]:
def train(d_model, g_model, gan_model, dataset, n_epochs=100, n_batch=1):
	# determine the output square shape of the discriminator
	n_patch = d_model.output_shape[1]
	# unpack dataset
	trainA, trainB = dataset
	# calculate the number of batches per training epoch
	bat_per_epo = int(len(trainA) / n_batch)
	# calculate the number of training iterations
	n_steps = bat_per_epo * n_epochs
	# manually enumerate epochs
	for i in range(n_steps):
		# select a batch of real samples
		[X_realA, X_realB], y_real = generate_real_samples(dataset, n_batch, n_patch)
		# generate a batch of fake samples
		X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
		# update discriminator for real samples
		d_loss1 = d_model.train_on_batch([X_realA, X_realB], y_real)
		# update discriminator for generated samples
		d_loss2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
		# update the generator
		g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
		# summarize performance
		print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
		# summarize model performance
		if (i) == n_steps-1:
			summarize_performance(i, g_model, dataset)        

In [ ]:
from numpy import load
def load_real_samples(brain, mask):
	# load the compressed arrays
    data = load(brain)
    print(data)
    data1 = load(mask)
    print(data1)
	# unpack the arrays
    X1, X2 = data['arr_0'], data1['arr_0']
	# scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]
dataset = load_real_samples(data_source1,data_source2)
print('Loaded', dataset[0].shape, dataset[1].shape)

# define input shape based on the loaded dataset
image_shape = (256,256,1)
print(image_shape)

# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# train model
train(d_model, g_model, gan_model, dataset, n_epochs = 100, n_batch=16)


In [ ]:
mypath = "../input/figshare-brain-tumor-dataset/dataset/data"
filelist = natsorted([f for f in listdir(mypath) if isfile(join(mypath, f))])

In [ ]:
filelist100 = filelist[100:200]

In [ ]:
path = "../input/figshare-brain-tumor-dataset/dataset/data/"
datalist = []
for i in range(0,100):
    f = h5py.File(path + filelist100[i],'r')
    list(f.keys())
    datalist.append(np.array(f.get('cjdata/tumorMask/')))
    datalist[i].reshape(512,512,1)
    datalist[i] = resize(datalist[i], (256, 256))
    
np.array(datalist)

In [ ]:
plt.imshow(datalist[0])

In [ ]:
kaamKaDataStack = tf.stack(
   datalist, axis=0, name='stack'
)

In [ ]:
np.savez("./test1.npz", kaamKaDataStack)

In [ ]:
path = "../input/figshare-brain-tumor-dataset/dataset/data/"
imageDatalist = []
for i in range(0,100):
    f1 = h5py.File(path + filelist100[i],'r')
    list(f1.keys())
    imageDatalist.append(np.array(f1.get('cjdata/image/')))
    imageDatalist[i].reshape(512,512,1)
    imageDatalist[i] = resize(imageDatalist[i], (256, 256))
    
np.array(imageDatalist)

In [ ]:
kaamKaImageStack = tf.stack(
   imageDatalist, axis=0, name='stack1'
)

In [ ]:
np.savez("./testimg1.npz", kaamKaImageStack)

In [ ]:
pwd

In [ ]:
from keras.models import load_model
from numpy import load
from numpy import vstack
from matplotlib import pyplot
from numpy.random import randint
 
# load and prepare training images
def load_real_samples(brain, mask):
	# load the compressed arrays
    data = load(brain)
    print(data)
    data1 = load(mask)
    print(data1)
	# unpack the arrays
    X1, X2 = data['arr_0'], data1['arr_0']
	# scale from [0,255] to [-1,1]
    X1 = (X1 - 127.5) / 127.5
    X2 = (X2 - 127.5) / 127.5
    return [X1, X2]
 
# plot source, generated and target images
def plot_images(src_img, gen_img, tar_img):
    images = vstack((src_img, gen_img, tar_img))
    # scale from [-1,1] to [0,1]
    images = (images + 1) / 2.0
    titles = ['Source', 'Generated', 'Expected']
    # plot images row by row
    for i in range(len(images)):
    # define subplot
        pyplot.subplot(1, 3, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(images[i])
        # show title
        pyplot.title(titles[i])
    pyplot.show()

# load dataset
[X1,X2] = load_real_samples('testimg1.npz','test1.npz')
#[X2] = load_real_samples('test1.npz')
print('Loaded', X1.shape, X2.shape)
# load model
model = load_model('model_000600.h5')
# select random example
ix = randint(0, len(X1), 1)
print(ix)
src_image, tar_image = X1[ix], X2[ix]
# generate image from source
gen_image = model.predict(src_image)
src_image = src_image.reshape(1,256,256,1) 
tar_image = tar_image.reshape(1,256,256,1)
# plot all three images
plot_images(src_image, gen_image, tar_image)